In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import matplotlib.pyplot as plt
from pydicom import dcmread
import tensorflow as tf
import cv2
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 255)
from sklearn.model_selection import train_test_split



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
i=0

for dirname, _, filenames in os.walk('/kaggle/input/ranzcr-clip-catheter-line-classification/train'):
    for filename in filenames: 
        i=i+1
        
print("The no of images in Train Dir",i)
i=0

for dirname, _, filenames in os.walk('/kaggle/input/ranzcr-clip-catheter-line-classification/test'):
    for filename in filenames: 
        i=i+1
        
print("The no of images in Test Dir",i)
i=0

train_data_dir = '/kaggle/input/ranzcr-clip-catheter-line-classification/train'
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
BASE_DIR = "/kaggle/input/ranzcr-clip-catheter-line-classification/"
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

In [ ]:
training_data = []

files =  os.listdir(train_data_dir)
sorted_files =  sorted(files)

def create_training_data():
    ii = 0
    for img in sorted_files:
        img_array = cv2.imread(os.path.join(train_data_dir,img), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (25,25))
        training_data.append(new_array)
        
create_training_data()

In [ ]:
#X = np.array(training_data).reshape(-1,256,256,1)
X = np.array(training_data)

df_train = df_train.sort_values(by=['StudyInstanceUID'])
y = df_train.drop(columns=['StudyInstanceUID','PatientID'],axis=1)
y = y.to_numpy()

print(X.shape)
print(y.shape)

In [ ]:
X = X.reshape(X.shape[0],25,25,1)

classes = ['ETT - Abnormal','ETT - Borderline','ETT - Normal','NGT - Abnormal','NGT - Borderline','NGT - Incompletely Imaged','NGT - Normal','CVC - Abnormal','CVC - Borderline','CVC - Normal','Swan Ganz Catheter Present']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

In [ ]:
num_filters = 20
filtersi = 5
poolsi = 2

model=Sequential()
model.add(Conv2D(num_filters, filtersi, strides = (1,1), input_shape = (25,25,1)))
model.add(MaxPooling2D(pool_size = poolsi))
model.add(Flatten())
model.add(Dense(10, activation = 'softmax'))

In [ ]:
model.compile('adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(X_train, to_categorical(np.argmax(y_train,axis=1)), epochs = 30, verbose = 1, validation_data = (X_test,to_categorical(np.argmax(y_test, axis=1))),batch_size=1000)

In [ ]:
score=model.evaluate(X_test,to_categorical(np.argmax(y_test,axis=1)),verbose=0)
print("test loss",score[0])
print("test accuracy",score[1])

In [ ]:
predictions = model.predict(X_test[:5])
print(to_categorical(np.argmax(predictions, axis=1)))
print(to_categorical(np.argmax(y_test, axis=1)))